# Logistic Regression

## Data Preprocessing

In [16]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
import random
import counting_fns as cf
from sklearn.utils import shuffle


# List of all the months
months = ['2_June', '3_July', '4_August', '5_September', '6_October']

# Define the path to the data directory and columns to keep
data_path = "/Users/mau/Library/CloudStorage/Dropbox/Mac/Documents/Dissertation/Chapter 2/Entire_Data/By month"

# Filter Columns
filter = ['session_time', 'gender', 'age_gen', 'day', 'timeofday', 'first_outcome',
        'first_wager','first_p/b', 'last_outcome', 'last_wager', 'last_p/b',
        'beginning_amt', 'ending_amt', 'ending_balance', 'ave_slotdenom', 
        'std_slotdenom', 'min_slotdenom', 'max_slotdenom', 'ave_theo_payback',
        'min_theo_payback', 'max_theo_payback', 'ave_wageramt', 'std_wageramt',
        'min_wager', 'max_wager', 'ave_p/b', 'std_p/b', 'max_p/b', 'max_profit', 'depletion_slope', 
        '#inc_slotdenom', '#dec_slotdenom', '#inc_maxbet', '#dec_maxbet', '#W', '#L', '#NH', '#D',
        'w/min', 'l/min', '#2ws', '2ws_profit', '2ws_wgramt','2ws/min', 
        '#3ws', '3ws_profit', '3ws_wgramt', '3ws/min', '#4ws', '4ws_profit', '4ws_wgramt', '4ws/min', 
        'w/g', 'l/g', 'nh/g', 'd/g', 'ave_time_per_gamble', 
        'min_time_per_gamble', 'max_time_per_gamble', 'total_gambles',
        'machines_changes', 'unique_machines', 'ave_time_per_machine', 'classification']

In [17]:
# Hyperparameters
Best_params_oct = {'C': 10, 'max_iter': 500, 'penalty': 'l2', 'solver': 'saga'}
Best_params_sep = {'C': 10, 'max_iter': 500, 'penalty': 'l1', 'solver': 'liblinear'}
Best_params_aug = {'C': 1, 'max_iter': 500, 'penalty': 'l2', 'solver': 'saga'}
Best_params_jul = {'C': 10, 'max_iter': 500, 'penalty': 'l2', 'solver': 'liblinear'}
Best_params_june =  {'C': 10, 'max_iter': 300, 'penalty': 'l1', 'solver': 'liblinear'}

# Best_params overall
best_params = {'C': 10, 'max_iter': 1500, 'penalty': 'l2', 'solver': 'lbfgs'}

## June

In [18]:
# Table creation 
months_train = ['4_August', '3_July', '5_September', '6_October']
months_test = ['2_June']
# Define time intervals
time_intervals = [1, 5, 10, 15]

# Initialize lists to store results
results = []
dataframes_features = {}
for time_interval in time_intervals:
    # Load dataset for the specific time interval
    file_name = f'{time_interval}min'
    # Load the datasets
    datasets = cf.load_and_preprocess_datasets_min_ntop(months, data_path, file_name, filter)

    # Create training and test sets
    dt_train = pd.concat([datasets['dtf'+month[1:]] for month in months_train])
    dt_test = pd.concat([datasets['dtf'+month[1:]] for month in months_test])

    # # Seperate dependent and independent variables
    X_train = dt_train.iloc[:, :-1].values
    y_train = dt_train.iloc[:, -1].values
    X_test = dt_test.iloc[:, :-1].values
    y_test = dt_test.iloc[:, -1].values

    # Econde gender column (Binary)
    le = LabelEncoder()

    # Binary Encode gender
    X_train[:, 0] = le.fit_transform(X_train[:, 0])
    X_test[:, 0] = le.transform(X_test[:, 0])

    # # # Encode age_generartion, first_outoce, last_outcome, time of day columns
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1, 2, 3, 4, 7])], remainder='passthrough')
    X_train = ct.fit_transform(X_train)
    X_test = ct.transform(X_test)

    y_train = le.fit_transform(y_train)
    y_test = le.fit_transform(y_test)

    sc = StandardScaler()

    # Scale all columns except the encoded ones
    X_train[:, 25:] = sc.fit_transform(X_train[:, 25:])
    X_test[:, 25:] = sc.transform(X_test[:, 25:])

    # Apply SMOTE to balance the dataset
    sm = SMOTE(random_state=42)
    X_train_resample, y_train_resample = sm.fit_resample(X_train, y_train)

    # Shuffle the resampled data
    X_train_resample, y_train_resample = shuffle(X_train_resample, y_train_resample, random_state=42)

    classifier = LogisticRegression(random_state = 0, **best_params)
    classifier.fit(X_train_resample, y_train_resample)

    y_pred = classifier.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    # Append results for this time interval
    results.append([f'{time_interval} min', round(accuracy, 3), round(precision, 3), round(recall, 3), round(f1, 3)])

    # drop last column
    dataset= dt_train.drop(['classification'], axis=1)

    # Get the feature names without 'remainder__' prefix
    feature_names = cf.get_feature_names_without_prefix(ct, input_features=dataset.columns)

    # Get the coefficients and feature names
    coefficients = classifier.coef_[0]

    # Create a DataFrame to display coefficients and feature names
    coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

    # Sort the coefficients by magnitude
    coefficients_df['Abs_Coefficient'] = np.abs(coefficients_df['Coefficient'])
    coefficients_df = coefficients_df.sort_values(by='Abs_Coefficient', ascending=False)

    dataframes_features[f'coefficients_df_{time_interval}min'] = coefficients_df
    

# Create a DataFrame for the results
columns = ['Time', 'Accuracy', 'Precision', 'Recall', 'F1 Score']
results_df = pd.DataFrame(results, columns=columns)

# Print the results as a table
# print(results_df)

# Print the results as a LaTeX table
latex_table = results_df.to_latex(index=False, escape=False)
print(latex_table)

\begin{tabular}{lrrrr}
\toprule
  Time &  Accuracy &  Precision &  Recall &  F1 Score \\
\midrule
 1 min &     0.588 &      0.514 &   0.534 &     0.460 \\
 5 min &     0.596 &      0.546 &   0.617 &     0.489 \\
10 min &     0.608 &      0.548 &   0.623 &     0.497 \\
15 min &     0.654 &      0.567 &   0.665 &     0.534 \\
\bottomrule
\end{tabular}



/var/folders/vp/1skwx2kd29s4fxnxx7tt9r6w0000gn/T/ipykernel_4286/3468090535.py:96: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = results_df.to_latex(index=False, escape=False)


In [19]:
# Top 5 features for each time interval
for time_interval in time_intervals:
    print(f'Top 5 features for {time_interval} min')
    print(dataframes_features[f'coefficients_df_{time_interval}min'].head(5))
    print('\n')

Top 5 features for 1 min
                   Feature  Coefficient  Abs_Coefficient
2   encoder__age_gen_Gen Z    -1.567702         1.567702
35           max_slotdenom    -1.540399         1.540399
34           min_slotdenom     1.188924         1.188924
37        min_theo_payback     1.054538         1.054538
36        ave_theo_payback    -0.862359         0.862359


Top 5 features for 5 min
                          Feature  Coefficient  Abs_Coefficient
2          encoder__age_gen_Gen Z    -1.400468         1.400468
4         encoder__age_gen_Silent     0.808003         0.808003
0   encoder__age_gen_Baby Boomers     0.579796         0.579796
3     encoder__age_gen_Millenials    -0.543512         0.543512
37               min_theo_payback     0.461409         0.461409


Top 5 features for 10 min
                          Feature  Coefficient  Abs_Coefficient
2          encoder__age_gen_Gen Z    -1.275883         1.275883
4         encoder__age_gen_Silent     0.753194         0.753194
32

In [20]:
# Define Path
os.chdir('/Users/mau/Library/CloudStorage/Dropbox/Mac/Documents/Dissertation/Chapter 2/Entire_Data/Feauture Importance/Minutes/LR/June')
# Save datafraes to csv files
for time_interval in time_intervals:
    print(f'Saving features for {time_interval} min interval')
    dataframes_features[f'coefficients_df_{time_interval}min'].to_csv(f'coefficients_df_{time_interval}min.csv')
    

Saving features for 1 min interval
Saving features for 5 min interval
Saving features for 10 min interval
Saving features for 15 min interval


## July

In [21]:
# Table creation 
months_train = ['4_August', '2_June', '5_September', '6_October']
months_test = ['3_July']
# Define time intervals
time_intervals = [1, 5, 10, 15]

# Initialize lists to store results
results = []
dataframes_features = {}
for time_interval in time_intervals:
    # Load dataset for the specific time interval
    file_name = f'{time_interval}min'
    # Load the datasets
    datasets = cf.load_and_preprocess_datasets_min_ntop(months, data_path, file_name, filter)

    # Create training and test sets
    dt_train = pd.concat([datasets['dtf'+month[1:]] for month in months_train])
    dt_test = pd.concat([datasets['dtf'+month[1:]] for month in months_test])

    # # Seperate dependent and independent variables
    X_train = dt_train.iloc[:, :-1].values
    y_train = dt_train.iloc[:, -1].values
    X_test = dt_test.iloc[:, :-1].values
    y_test = dt_test.iloc[:, -1].values

    # Econde gender column (Binary)
    le = LabelEncoder()

    # Binary Encode gender
    X_train[:, 0] = le.fit_transform(X_train[:, 0])
    X_test[:, 0] = le.transform(X_test[:, 0])

    # # # Encode age_generartion, first_outoce, last_outcome, time of day columns
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1, 2, 3, 4, 7])], remainder='passthrough')
    X_train = ct.fit_transform(X_train)
    X_test = ct.transform(X_test)

    y_train = le.fit_transform(y_train)
    y_test = le.fit_transform(y_test)

    sc = StandardScaler()

    # Scale all columns except the encoded ones
    X_train[:, 25:] = sc.fit_transform(X_train[:, 25:])
    X_test[:, 25:] = sc.transform(X_test[:, 25:])

    # Apply SMOTE to balance the dataset
    sm = SMOTE(random_state=42)
    X_train_resample, y_train_resample = sm.fit_resample(X_train, y_train)

    # Shuffle the resampled data
    X_train_resample, y_train_resample = shuffle(X_train_resample, y_train_resample, random_state=42)

    classifier = LogisticRegression(random_state = 0, **best_params)
    classifier.fit(X_train_resample, y_train_resample)

    y_pred = classifier.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    # Append results for this time interval
    results.append([f'{time_interval} min', round(accuracy, 3), round(precision, 3), round(recall, 3), round(f1, 3)])

    # drop last column
    dataset= dt_train.drop(['classification'], axis=1)

    # Get the feature names without 'remainder__' prefix
    feature_names = cf.get_feature_names_without_prefix(ct, input_features=dataset.columns)

    # Get the coefficients and feature names
    coefficients = classifier.coef_[0]

    # Create a DataFrame to display coefficients and feature names
    coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

    # Sort the coefficients by magnitude
    coefficients_df['Abs_Coefficient'] = np.abs(coefficients_df['Coefficient'])
    coefficients_df = coefficients_df.sort_values(by='Abs_Coefficient', ascending=False)

    dataframes_features[f'coefficients_df_{time_interval}min'] = coefficients_df
    

# Create a DataFrame for the results
columns = ['Time', 'Accuracy', 'Precision', 'Recall', 'F1 Score']
results_df = pd.DataFrame(results, columns=columns)

# Print the results as a table
# print(results_df)

# Print the results as a LaTeX table
latex_table = results_df.to_latex(index=False, escape=False)
print(latex_table)

\begin{tabular}{lrrrr}
\toprule
  Time &  Accuracy &  Precision &  Recall &  F1 Score \\
\midrule
 1 min &     0.636 &      0.549 &   0.626 &     0.507 \\
 5 min &     0.675 &      0.565 &   0.663 &     0.539 \\
10 min &     0.701 &      0.579 &   0.693 &     0.563 \\
15 min &     0.719 &      0.585 &   0.701 &     0.576 \\
\bottomrule
\end{tabular}



/var/folders/vp/1skwx2kd29s4fxnxx7tt9r6w0000gn/T/ipykernel_4286/569076621.py:96: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = results_df.to_latex(index=False, escape=False)


In [22]:
# Top 5 features for each time interval
for time_interval in time_intervals:
    print(f'Top 5 features for {time_interval} min')
    print(dataframes_features[f'coefficients_df_{time_interval}min'].head(5))
    print('\n')

Top 5 features for 1 min
                          Feature  Coefficient  Abs_Coefficient
2          encoder__age_gen_Gen Z    -1.384011         1.384011
4         encoder__age_gen_Silent     0.787211         0.787211
0   encoder__age_gen_Baby Boomers     0.668812         0.668812
3     encoder__age_gen_Millenials    -0.602106         0.602106
42                      max_wager     0.555044         0.555044


Top 5 features for 5 min
                          Feature  Coefficient  Abs_Coefficient
2          encoder__age_gen_Gen Z    -1.192791         1.192791
4         encoder__age_gen_Silent     0.790155         0.790155
3     encoder__age_gen_Millenials    -0.633203         0.633203
0   encoder__age_gen_Baby Boomers     0.551558         0.551558
37               min_theo_payback     0.533512         0.533512


Top 5 features for 10 min
                        Feature  Coefficient  Abs_Coefficient
2        encoder__age_gen_Gen Z    -1.031941         1.031941
4       encoder__age_gen_Sil

In [23]:
# Define Path
os.chdir('/Users/mau/Library/CloudStorage/Dropbox/Mac/Documents/Dissertation/Chapter 2/Entire_Data/Feauture Importance/Minutes/LR/July')
# Save datafraes to csv files
for time_interval in time_intervals:
    print(f'Saving features for {time_interval} min interval')
    dataframes_features[f'coefficients_df_{time_interval}min'].to_csv(f'coefficients_df_{time_interval}min.csv')
    

Saving features for 1 min interval
Saving features for 5 min interval
Saving features for 10 min interval
Saving features for 15 min interval


## August

In [24]:
# Table creation 
months_train = ['2_June', '3_July', '5_September', '6_October']
months_test = ['4_August']
# Define time intervals
time_intervals = [1, 5, 10, 15]

# Initialize lists to store results
results = []
dataframes_features = {}
for time_interval in time_intervals:
    # Load dataset for the specific time interval
    file_name = f'{time_interval}min'
    # Load the datasets
    datasets = cf.load_and_preprocess_datasets_min_ntop(months, data_path, file_name, filter)

    # Create training and test sets
    dt_train = pd.concat([datasets['dtf'+month[1:]] for month in months_train])
    dt_test = pd.concat([datasets['dtf'+month[1:]] for month in months_test])

    # # Seperate dependent and independent variables
    X_train = dt_train.iloc[:, :-1].values
    y_train = dt_train.iloc[:, -1].values
    X_test = dt_test.iloc[:, :-1].values
    y_test = dt_test.iloc[:, -1].values

    # Econde gender column (Binary)
    le = LabelEncoder()

    # Binary Encode gender
    X_train[:, 0] = le.fit_transform(X_train[:, 0])
    X_test[:, 0] = le.transform(X_test[:, 0])

    # # # Encode age_generartion, first_outoce, last_outcome, time of day columns
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1, 2, 3, 4, 7])], remainder='passthrough')
    X_train = ct.fit_transform(X_train)
    X_test = ct.transform(X_test)

    y_train = le.fit_transform(y_train)
    y_test = le.fit_transform(y_test)

    sc = StandardScaler()

    # Scale all columns except the encoded ones
    X_train[:, 25:] = sc.fit_transform(X_train[:, 25:])
    X_test[:, 25:] = sc.transform(X_test[:, 25:])

    # Apply SMOTE to balance the dataset
    sm = SMOTE(random_state=42)
    X_train_resample, y_train_resample = sm.fit_resample(X_train, y_train)

    # Shuffle the resampled data
    X_train_resample, y_train_resample = shuffle(X_train_resample, y_train_resample, random_state=42)

    classifier = LogisticRegression(random_state = 0, **best_params)
    classifier.fit(X_train_resample, y_train_resample)

    y_pred = classifier.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    # Append results for this time interval
    results.append([f'{time_interval} min', round(accuracy, 3), round(precision, 3), round(recall, 3), round(f1, 3)])

    # drop last column
    dataset= dt_train.drop(['classification'], axis=1)

    # Get the feature names without 'remainder__' prefix
    feature_names = cf.get_feature_names_without_prefix(ct, input_features=dataset.columns)

    # Get the coefficients and feature names
    coefficients = classifier.coef_[0]

    # Create a DataFrame to display coefficients and feature names
    coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

    # Sort the coefficients by magnitude
    coefficients_df['Abs_Coefficient'] = np.abs(coefficients_df['Coefficient'])
    coefficients_df = coefficients_df.sort_values(by='Abs_Coefficient', ascending=False)

    dataframes_features[f'coefficients_df_{time_interval}min'] = coefficients_df
    

# Create a DataFrame for the results
columns = ['Time', 'Accuracy', 'Precision', 'Recall', 'F1 Score']
results_df = pd.DataFrame(results, columns=columns)

# Print the results as a table
# print(results_df)

# Print the results as a LaTeX table
latex_table = results_df.to_latex(index=False, escape=False)
print(latex_table)

\begin{tabular}{lrrrr}
\toprule
  Time &  Accuracy &  Precision &  Recall &  F1 Score \\
\midrule
 1 min &     0.643 &      0.548 &   0.625 &     0.510 \\
 5 min &     0.671 &      0.565 &   0.665 &     0.537 \\
10 min &     0.699 &      0.578 &   0.692 &     0.561 \\
15 min &     0.719 &      0.586 &   0.706 &     0.577 \\
\bottomrule
\end{tabular}



/var/folders/vp/1skwx2kd29s4fxnxx7tt9r6w0000gn/T/ipykernel_4286/3826216220.py:96: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = results_df.to_latex(index=False, escape=False)


In [25]:
# Top 5 features for each time interval
for time_interval in time_intervals:
    print(f'Top 5 features for {time_interval} min')
    print(dataframes_features[f'coefficients_df_{time_interval}min'].head(5))
    print('\n')

Top 5 features for 1 min
                   Feature  Coefficient  Abs_Coefficient
35           max_slotdenom    -3.801221         3.801221
34           min_slotdenom     2.425592         2.425592
2   encoder__age_gen_Gen Z    -2.093972         2.093972
42               max_wager    -1.589277         1.589277
25             first_wager     1.320515         1.320515


Top 5 features for 5 min
                          Feature  Coefficient  Abs_Coefficient
2          encoder__age_gen_Gen Z    -1.891316         1.891316
4         encoder__age_gen_Silent     0.978307         0.978307
0   encoder__age_gen_Baby Boomers     0.652839         0.652839
41                      min_wager     0.440398         0.440398
3     encoder__age_gen_Millenials    -0.394435         0.394435


Top 5 features for 10 min
                          Feature  Coefficient  Abs_Coefficient
2          encoder__age_gen_Gen Z    -1.850159         1.850159
4         encoder__age_gen_Silent     0.956828         0.956828
0 

In [26]:
# Define Path
os.chdir('/Users/mau/Library/CloudStorage/Dropbox/Mac/Documents/Dissertation/Chapter 2/Entire_Data/Feauture Importance/Minutes/LR/August')
# Save datafraes to csv files
for time_interval in time_intervals:
    print(f'Saving features for {time_interval} min interval')
    dataframes_features[f'coefficients_df_{time_interval}min'].to_csv(f'coefficients_df_{time_interval}min.csv')
    

Saving features for 1 min interval
Saving features for 5 min interval
Saving features for 10 min interval
Saving features for 15 min interval


## September

In [27]:
# Table creation 
months_train = ['2_June', '3_July', '4_August', '6_October']
months_test = ['5_September']
# Define time intervals
time_intervals = [1, 5, 10, 15]

# Initialize lists to store results
results = []
dataframes_features = {}
for time_interval in time_intervals:
    # Load dataset for the specific time interval
    file_name = f'{time_interval}min'
    # Load the datasets
    datasets = cf.load_and_preprocess_datasets_min_ntop(months, data_path, file_name, filter)

    # Create training and test sets
    dt_train = pd.concat([datasets['dtf'+month[1:]] for month in months_train])
    dt_test = pd.concat([datasets['dtf'+month[1:]] for month in months_test])

    # # Seperate dependent and independent variables
    X_train = dt_train.iloc[:, :-1].values
    y_train = dt_train.iloc[:, -1].values
    X_test = dt_test.iloc[:, :-1].values
    y_test = dt_test.iloc[:, -1].values

    # Econde gender column (Binary)
    le = LabelEncoder()

    # Binary Encode gender
    X_train[:, 0] = le.fit_transform(X_train[:, 0])
    X_test[:, 0] = le.transform(X_test[:, 0])

    # # # Encode age_generartion, first_outoce, last_outcome, time of day columns
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1, 2, 3, 4, 7])], remainder='passthrough')
    X_train = ct.fit_transform(X_train)
    X_test = ct.transform(X_test)

    y_train = le.fit_transform(y_train)
    y_test = le.fit_transform(y_test)

    sc = StandardScaler()

    # Scale all columns except the encoded ones
    X_train[:, 25:] = sc.fit_transform(X_train[:, 25:])
    X_test[:, 25:] = sc.transform(X_test[:, 25:])

    # Apply SMOTE to balance the dataset
    sm = SMOTE(random_state=42)
    X_train_resample, y_train_resample = sm.fit_resample(X_train, y_train)

    # Shuffle the resampled data
    X_train_resample, y_train_resample = shuffle(X_train_resample, y_train_resample, random_state=42)

    classifier = LogisticRegression(random_state = 0, **best_params)
    classifier.fit(X_train_resample, y_train_resample)

    y_pred = classifier.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    # Append results for this time interval
    results.append([f'{time_interval} min', round(accuracy, 3), round(precision, 3), round(recall, 3), round(f1, 3)])

    # drop last column
    dataset= dt_train.drop(['classification'], axis=1)

    # Get the feature names without 'remainder__' prefix
    feature_names = cf.get_feature_names_without_prefix(ct, input_features=dataset.columns)

    # Get the coefficients and feature names
    coefficients = classifier.coef_[0]

    # Create a DataFrame to display coefficients and feature names
    coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

    # Sort the coefficients by magnitude
    coefficients_df['Abs_Coefficient'] = np.abs(coefficients_df['Coefficient'])
    coefficients_df = coefficients_df.sort_values(by='Abs_Coefficient', ascending=False)

    dataframes_features[f'coefficients_df_{time_interval}min'] = coefficients_df
    

# Create a DataFrame for the results
columns = ['Time', 'Accuracy', 'Precision', 'Recall', 'F1 Score']
results_df = pd.DataFrame(results, columns=columns)

# Print the results as a table
# print(results_df)

# Print the results as a LaTeX table
latex_table = results_df.to_latex(index=False, escape=False)
print(latex_table)

\begin{tabular}{lrrrr}
\toprule
  Time &  Accuracy &  Precision &  Recall &  F1 Score \\
\midrule
 1 min &     0.612 &      0.549 &   0.628 &     0.497 \\
 5 min &     0.647 &      0.563 &   0.663 &     0.525 \\
10 min &     0.674 &      0.571 &   0.677 &     0.544 \\
15 min &     0.693 &      0.580 &   0.696 &     0.561 \\
\bottomrule
\end{tabular}



/var/folders/vp/1skwx2kd29s4fxnxx7tt9r6w0000gn/T/ipykernel_4286/3963048803.py:96: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = results_df.to_latex(index=False, escape=False)


In [28]:
# Top 5 features for each time interval
for time_interval in time_intervals:
    print(f'Top 5 features for {time_interval} min')
    print(dataframes_features[f'coefficients_df_{time_interval}min'].head(5))
    print('\n')

Top 5 features for 1 min
                   Feature  Coefficient  Abs_Coefficient
36        ave_theo_payback    -2.416816         2.416816
37        min_theo_payback     1.853085         1.853085
2   encoder__age_gen_Gen Z    -1.499370         1.499370
35           max_slotdenom    -1.256808         1.256808
34           min_slotdenom     1.122604         1.122604


Top 5 features for 5 min
                          Feature  Coefficient  Abs_Coefficient
2          encoder__age_gen_Gen Z    -1.283685         1.283685
4         encoder__age_gen_Silent     0.700387         0.700387
0   encoder__age_gen_Baby Boomers     0.563621         0.563621
3     encoder__age_gen_Millenials    -0.528091         0.528091
74            ave_time_per_gamble    -0.409795         0.409795


Top 5 features for 10 min
                        Feature  Coefficient  Abs_Coefficient
2        encoder__age_gen_Gen Z    -1.194266         1.194266
36             ave_theo_payback     1.057104         1.057104
4       

In [29]:
# Define Path
os.chdir('/Users/mau/Library/CloudStorage/Dropbox/Mac/Documents/Dissertation/Chapter 2/Entire_Data/Feauture Importance/Minutes/LR/September')
# Save datafraes to csv files
for time_interval in time_intervals:
    print(f'Saving features for {time_interval} min interval')
    dataframes_features[f'coefficients_df_{time_interval}min'].to_csv(f'coefficients_df_{time_interval}min.csv')
    

Saving features for 1 min interval
Saving features for 5 min interval
Saving features for 10 min interval
Saving features for 15 min interval


## October

In [30]:
# Table creation 
months_train = ['2_June', '3_July', '4_August', '5_September']
months_test = ['6_October']
# Define time intervals
time_intervals = [1, 5, 10, 15]

# Initialize lists to store results
results = []
dataframes_features = {}
for time_interval in time_intervals:
    # Load dataset for the specific time interval
    file_name = f'{time_interval}min'
    # Load the datasets
    datasets = cf.load_and_preprocess_datasets_min_ntop(months, data_path, file_name, filter)

    # Create training and test sets
    dt_train = pd.concat([datasets['dtf'+month[1:]] for month in months_train])
    dt_test = pd.concat([datasets['dtf'+month[1:]] for month in months_test])

    # # Seperate dependent and independent variables
    X_train = dt_train.iloc[:, :-1].values
    y_train = dt_train.iloc[:, -1].values
    X_test = dt_test.iloc[:, :-1].values
    y_test = dt_test.iloc[:, -1].values

    # Econde gender column (Binary)
    le = LabelEncoder()

    # Binary Encode gender
    X_train[:, 0] = le.fit_transform(X_train[:, 0])
    X_test[:, 0] = le.transform(X_test[:, 0])

    # # # Encode age_generartion, first_outoce, last_outcome, time of day columns
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1, 2, 3, 4, 7])], remainder='passthrough')
    X_train = ct.fit_transform(X_train)
    X_test = ct.transform(X_test)

    y_train = le.fit_transform(y_train)
    y_test = le.fit_transform(y_test)

    sc = StandardScaler()

    # Scale all columns except the encoded ones
    X_train[:, 25:] = sc.fit_transform(X_train[:, 25:])
    X_test[:, 25:] = sc.transform(X_test[:, 25:])

    # Apply SMOTE to balance the dataset
    sm = SMOTE(random_state=42)
    X_train_resample, y_train_resample = sm.fit_resample(X_train, y_train)

    # Shuffle the resampled data
    X_train_resample, y_train_resample = shuffle(X_train_resample, y_train_resample, random_state=42)

    classifier = LogisticRegression(random_state = 0, **best_params)
    classifier.fit(X_train_resample, y_train_resample)

    y_pred = classifier.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    # Append results for this time interval
    results.append([f'{time_interval} min', round(accuracy, 3), round(precision, 3), round(recall, 3), round(f1, 3)])

    # drop last column
    dataset= dt_train.drop(['classification'], axis=1)

    # Get the feature names without 'remainder__' prefix
    feature_names = cf.get_feature_names_without_prefix(ct, input_features=dataset.columns)

    # Get the coefficients and feature names
    coefficients = classifier.coef_[0]

    # Create a DataFrame to display coefficients and feature names
    coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

    # Sort the coefficients by magnitude
    coefficients_df['Abs_Coefficient'] = np.abs(coefficients_df['Coefficient'])
    coefficients_df = coefficients_df.sort_values(by='Abs_Coefficient', ascending=False)

    dataframes_features[f'coefficients_df_{time_interval}min'] = coefficients_df
    

# Create a DataFrame for the results
columns = ['Time', 'Accuracy', 'Precision', 'Recall', 'F1 Score']
results_df = pd.DataFrame(results, columns=columns)

# Print the results as a table
# print(results_df)

# Print the results as a LaTeX table
latex_table = results_df.to_latex(index=False, escape=False)
print(latex_table)

\begin{tabular}{lrrrr}
\toprule
  Time &  Accuracy &  Precision &  Recall &  F1 Score \\
\midrule
 1 min &     0.573 &      0.543 &   0.618 &     0.472 \\
 5 min &     0.615 &      0.555 &   0.649 &     0.503 \\
10 min &     0.648 &      0.567 &   0.677 &     0.528 \\
15 min &     0.675 &      0.575 &   0.693 &     0.548 \\
\bottomrule
\end{tabular}



/var/folders/vp/1skwx2kd29s4fxnxx7tt9r6w0000gn/T/ipykernel_4286/3367126610.py:96: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = results_df.to_latex(index=False, escape=False)


In [31]:
# Top 5 features for each time interval
for time_interval in time_intervals:
    print(f'Top 5 features for {time_interval} min')
    print(dataframes_features[f'coefficients_df_{time_interval}min'].head(5))
    print('\n')

Top 5 features for 1 min
                   Feature  Coefficient  Abs_Coefficient
2   encoder__age_gen_Gen Z    -1.557634         1.557634
38        max_theo_payback    -1.529084         1.529084
35           max_slotdenom    -1.224510         1.224510
37        min_theo_payback     1.208374         1.208374
39            ave_wageramt    -0.872416         0.872416


Top 5 features for 5 min
                          Feature  Coefficient  Abs_Coefficient
2          encoder__age_gen_Gen Z    -1.386665         1.386665
4         encoder__age_gen_Silent     0.765910         0.765910
37               min_theo_payback     0.685957         0.685957
0   encoder__age_gen_Baby Boomers     0.586649         0.586649
3     encoder__age_gen_Millenials    -0.555190         0.555190


Top 5 features for 10 min
                          Feature  Coefficient  Abs_Coefficient
2          encoder__age_gen_Gen Z    -1.229565         1.229565
4         encoder__age_gen_Silent     0.732307         0.732307
3 

In [32]:
# Define Path
os.chdir('/Users/mau/Library/CloudStorage/Dropbox/Mac/Documents/Dissertation/Chapter 2/Entire_Data/Feauture Importance/Minutes/LR/October')
# Save datafraes to csv files
for time_interval in time_intervals:
    print(f'Saving features for {time_interval} min interval')
    dataframes_features[f'coefficients_df_{time_interval}min'].to_csv(f'coefficients_df_{time_interval}min.csv')
    

Saving features for 1 min interval
Saving features for 5 min interval
Saving features for 10 min interval
Saving features for 15 min interval
